In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import geopandas as gpd
import pickle

In [2]:
with open('nash_hosp_referrers.pickle', 'rb') as file:
    nash_hosp_referrers = pickle.load(file)

In [3]:
# get total out of state referral counts to Nashville, TN hospitals for choropleth
state_counts = nash_hosp_referrers.groupby('location_address_state_name').agg(
    referrals=('from_npi', 'nunique'),
    total_transactions=('transaction_count', 'sum')
).reset_index()
state_counts

,location_address_state_name,referrals,total_transactions
0,AL,18,3806
1,AR,4,580
2,AZ,2,1467
3,CA,13,3810
4,CO,10,2216
5,CT,7,3025
6,DC,3,602
7,DE,1,471
8,FL,27,6934
9,GA,15,4269


In [5]:
# filter state of TN for choropleth
nash_hosp_tn = nash_hosp_referrers.loc[nash_hosp_referrers['location_address_state_name'] == 'TN']
nash_hosp_tn = nash_hosp_tn[['location_address_postal_code', 'from_npi', 'transaction_count']].groupby(['location_address_postal_code']).agg({'transaction_count':'sum', 'from_npi':'nunique', 'location_address_postal_code':'count'}).rename(columns={'location_address_postal_code':'referrals'}).reset_index()
nash_hosp_tn

,location_address_postal_code,transaction_count,from_npi,referrals
0,37013,2406,10,14
1,37015,848,3,7
2,37025,169,1,1
3,37027,13259,59,77
4,37030,207,1,2
...,...,...,...,...
117,38551,60,1,1
118,38555,1160,14,15
119,38556,203,3,3
120,38570,303,3,3


In [6]:
# read in USA shapefile
us_states = gpd.read_file('cb_2018_us_state_500k.shp')

# merge shapefile with state_counts df
state_map = us_states.merge(state_counts, left_on='STUSPS', right_on='location_address_state_name')
state_map

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,location_address_state_name,referrals,total_transactions
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21523, -88.49176 ...",MS,6,2556
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ...",NC,26,10206
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-103.00257 36.52659, -103.00219 36.6...",OK,1,92
3,51,01779803,0400000US51,51,VA,Virginia,00,102257717110,8528531774,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ...",VA,9,2485
4,54,01779805,0400000US54,54,WV,West Virginia,00,62266474513,489028543,"POLYGON ((-82.64320 38.16909, -82.64300 38.169...",WV,2,1515
5,22,01629543,0400000US22,22,LA,Louisiana,00,111897594374,23753621895,"MULTIPOLYGON (((-88.86770 29.86155, -88.86566 ...",LA,10,3886
6,26,01779789,0400000US26,26,MI,Michigan,00,146600952990,103885855702,"MULTIPOLYGON (((-83.19159 42.03537, -83.18993 ...",MI,26,2990
7,25,00606926,0400000US25,25,MA,Massachusetts,00,20205125364,7129925486,"MULTIPOLYGON (((-70.23405 41.28565, -70.22361 ...",MA,12,2516
8,16,01779783,0400000US16,16,ID,Idaho,00,214049787659,2391722557,"POLYGON ((-117.24267 44.39655, -117.23484 44.3...",ID,1,106
9,12,00294478,0400000US12,12,FL,Florida,00,138949136250,31361101223,"MULTIPOLYGON (((-80.17628 25.52505, -80.17395 ...",FL,27,6934


In [7]:
# delete TN from df in order to not skew choropleth
state_map_tn = state_map.drop(21)

In [8]:
# read in zip code shapefile
gdf = gpd.read_file('cb_2018_us_zcta510_500k.shp')

# read in zip code to state reference file
zip_to_state = pd.read_csv('zcta_county_rel_10.txt', dtype=str, delimiter=',')[['ZCTA5','STATE']]

# merge shapefile with zip to state mapping
gdf = gdf.merge(zip_to_state, left_on='ZCTA5CE10', right_on='ZCTA5')

# drop unnecessary columns and rename STATE column
gdf = gdf.drop(['ZCTA5'], axis=1).rename(columns={'STATE': 'STUSPS'})

# Create a dictionary to map numerical codes to state abbreviations
state_codes = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '60': 'AS', '66': 'GU', '69': 'MP', '72': 'PR', '78': 'VI'}

# Convert the numerical codes to state abbreviations
gdf['STUSPS'] = gdf['STUSPS'].astype(str).str.zfill(2).map(state_codes)
gdf

,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10,geometry,STUSPS
0,36083,8600000US36083,36083,659750662,5522919,"MULTIPOLYGON (((-85.63225 32.28098, -85.62439 ...",AL
1,35441,8600000US35441,35441,172850429,8749105,"MULTIPOLYGON (((-87.83287 32.84437, -87.83184 ...",AL
2,35051,8600000US35051,35051,280236456,5427285,"POLYGON ((-86.74384 33.25002, -86.73802 33.251...",AL
3,35121,8600000US35121,35121,372736030,5349303,"POLYGON ((-86.58527 33.94743, -86.58033 33.948...",AL
4,35121,8600000US35121,35121,372736030,5349303,"POLYGON ((-86.58527 33.94743, -86.58033 33.948...",AL
...,...,...,...,...,...,...,...
44405,50460,8600000US50460,50460,93166133,0,"POLYGON ((-92.80629 43.23026, -92.80354 43.232...",IA
44406,50460,8600000US50460,50460,93166133,0,"POLYGON ((-92.80629 43.23026, -92.80354 43.232...",IA
44407,40870,8600000US40870,40870,18226594,201441,"POLYGON ((-83.19264 36.91650, -83.19086 36.916...",KY
44408,40914,8600000US40914,40914,32269366,419039,"POLYGON ((-83.62748 37.07419, -83.62455 37.073...",KY


In [9]:
# filter for just TN
tn_gdf = gdf[gdf['STUSPS'] == 'TN']
tn_gdf

,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10,geometry,STUSPS
312,37066,8600000US37066,37066,296571031,15770388,"POLYGON ((-86.58467 36.41678, -86.57109 36.412...",TN
313,38301,8600000US38301,38301,427555600,617738,"POLYGON ((-89.07140 35.63486, -89.07094 35.640...",TN
314,37174,8600000US37174,37174,177635552,219653,"MULTIPOLYGON (((-86.90211 35.76941, -86.89948 ...",TN
315,37174,8600000US37174,37174,177635552,219653,"MULTIPOLYGON (((-86.90211 35.76941, -86.89948 ...",TN
558,37338,8600000US37338,37338,195021395,199083,"POLYGON ((-85.33219 35.37360, -85.33062 35.380...",TN
...,...,...,...,...,...,...,...
44136,38547,8600000US38547,38547,84906105,119487,"MULTIPOLYGON (((-86.00408 36.21457, -86.00144 ...",TN
44145,38553,8600000US38553,38553,124094916,0,"POLYGON ((-85.11958 36.14472, -85.10585 36.163...",TN
44305,37323,8600000US37323,37323,309435474,285564,"POLYGON ((-84.94033 34.99622, -84.93666 35.004...",TN
44306,37323,8600000US37323,37323,309435474,285564,"POLYGON ((-84.94033 34.99622, -84.93666 35.004...",TN


In [10]:
# merge TN hospital referrals with zip code shapefile
tn_map = tn_gdf.merge(nash_hosp_tn, left_on='ZCTA5CE10', right_on='location_address_postal_code')
tn_map

,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10,geometry,STUSPS,location_address_postal_code,transaction_count,from_npi,referrals
0,37066,8600000US37066,37066,296571031,15770388,"POLYGON ((-86.58467 36.41678, -86.57109 36.412...",TN,37066,5501,27,41
1,38301,8600000US38301,38301,427555600,617738,"POLYGON ((-89.07140 35.63486, -89.07094 35.640...",TN,38301,592,8,8
2,37174,8600000US37174,37174,177635552,219653,"MULTIPOLYGON (((-86.90211 35.76941, -86.89948 ...",TN,37174,4174,17,22
3,37174,8600000US37174,37174,177635552,219653,"MULTIPOLYGON (((-86.90211 35.76941, -86.89948 ...",TN,37174,4174,17,22
4,37064,8600000US37064,37064,462538067,690730,"MULTIPOLYGON (((-86.80055 35.90413, -86.79779 ...",TN,37064,3367,19,22
...,...,...,...,...,...,...,...,...,...,...,...
192,38374,8600000US38374,38374,114176971,0,"POLYGON ((-88.29129 35.59902, -88.28292 35.598...",TN,38374,51,1,1
193,38374,8600000US38374,38374,114176971,0,"POLYGON ((-88.29129 35.59902, -88.28292 35.598...",TN,38374,51,1,1
194,37923,8600000US37923,37923,28981713,33160,"MULTIPOLYGON (((-84.06589 35.94811, -84.06552 ...",TN,37923,1014,4,4
195,37128,8600000US37128,37128,139683809,158520,"POLYGON ((-86.60066 35.84210, -86.60102 35.844...",TN,37128,485,3,5


In [11]:
# repeat process for KY
ky_gdf = gdf[gdf['STUSPS'] == 'KY']
ky_gdf

,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10,geometry,STUSPS
231,41073,8600000US41073,41073,2430131,493999,"POLYGON ((-84.49355 39.10272, -84.48774 39.110...",KY
234,42464,8600000US42464,42464,158082090,4724356,"POLYGON ((-87.45777 37.24847, -87.45273 37.251...",KY
235,42464,8600000US42464,42464,158082090,4724356,"POLYGON ((-87.45777 37.24847, -87.45273 37.251...",KY
236,42464,8600000US42464,42464,158082090,4724356,"POLYGON ((-87.45777 37.24847, -87.45273 37.251...",KY
237,41083,8600000US41083,41083,88943841,1476043,"MULTIPOLYGON (((-84.92561 38.76279, -84.92546 ...",KY
...,...,...,...,...,...,...,...
44380,42082,8600000US42082,42082,73138739,1600061,"POLYGON ((-88.55637 36.90836, -88.55484 36.910...",KY
44381,42082,8600000US42082,42082,73138739,1600061,"POLYGON ((-88.55637 36.90836, -88.55484 36.910...",KY
44382,42082,8600000US42082,42082,73138739,1600061,"POLYGON ((-88.55637 36.90836, -88.55484 36.910...",KY
44407,40870,8600000US40870,40870,18226594,201441,"POLYGON ((-83.19264 36.91650, -83.19086 36.916...",KY


In [12]:
nash_hosp_ky = nash_hosp_referrers.loc[nash_hosp_referrers['location_address_state_name'] == 'KY']
nash_hosp_ky

,transaction_count,hospital,location_address_city_name,location_address_state_name,location_address_postal_code,specialty,from_npi,first_name,last_name,credential,hosp_top_referrals
55,63,"HCA HEALTH SERVICES OF TENNESSEE, INC.",BOWLING GREEN,KY,42104,Diagnostic Radiology Physician,1477542942,KEVIN,BURNER,M.D.,943.0
85,63,"HCA HEALTH SERVICES OF TENNESSEE, INC.",LEXINGTON,KY,40503,Internal Medicine Physician,1730345505,SHEFALI,PARANJAPE,MD,943.0
325,204,SAINT THOMAS WEST HOSPITAL,FORT CAMPBELL,KY,42223,Family Nurse Practitioner,1275780298,ZIPPORAH,DAVIS,"MSN, RN, NP-C",485.5
362,76,SAINT THOMAS WEST HOSPITAL,HOPKINSVILLE,KY,42240,Diagnostic Radiology Physician,1346254919,CYRUS,CHAPMAN,MD,904.0
804,87,SAINT THOMAS WEST HOSPITAL,MADISONVILLE,KY,42431,Dermatology Physician,1538108873,WILLIAM,SMITH,M.D.,829.0
...,...,...,...,...,...,...,...,...,...,...,...
5080,146,VANDERBILT UNIVERSITY MEDICAL CENTER,PADUCAH,KY,42003,Medical Oncology Physician,1376657254,WILLIAM,SKINNER,M.D.,1137.0
5134,125,VANDERBILT UNIVERSITY MEDICAL CENTER,HOPKINSVILLE,KY,42240,Hematology & Oncology Physician,1396744033,RAMESH,PATEL,M.D.,1301.0
5149,77,VANDERBILT UNIVERSITY MEDICAL CENTER,PADUCAH,KY,42003,Radiation Oncology Physician,1396837159,PETER,LOCKEN,M.D.,1907.0
5172,53,VANDERBILT UNIVERSITY MEDICAL CENTER,HOPKINSVILLE,KY,42240,Family Medicine Physician,1407149560,BILLY,FRALISH,MD,2474.0


In [13]:
nash_hosp_ky = nash_hosp_ky[['location_address_postal_code', 'from_npi', 'transaction_count']].groupby(['location_address_postal_code']).agg({'transaction_count':'sum', 'from_npi':'nunique', 'location_address_postal_code':'count'}).rename(columns={'location_address_postal_code':'referrals'}).reset_index()
nash_hosp_ky

,location_address_postal_code,transaction_count,from_npi,referrals
0,40202,774,2,2
1,40204,197,1,1
2,40207,340,2,2
3,40216,1479,2,2
4,40502,145,1,1
5,40503,982,1,2
6,40536,235,2,2
7,41042,64,1,1
8,42001,597,7,7
9,42003,2112,24,24


In [14]:
ky_map = ky_gdf.merge(nash_hosp_ky, left_on='ZCTA5CE10', right_on='location_address_postal_code')
ky_map

,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10,geometry,STUSPS,location_address_postal_code,transaction_count,from_npi,referrals
0,42431,8600000US42431,42431,431514537,15296948,"POLYGON ((-87.64934 37.31273, -87.64502 37.312...",KY,42431,199,3,3
1,42071,8600000US42071,42071,549435948,29707707,"POLYGON ((-88.49083 36.63317, -88.48740 36.633...",KY,42071,963,10,10
2,42071,8600000US42071,42071,549435948,29707707,"POLYGON ((-88.49083 36.63317, -88.48740 36.633...",KY,42071,963,10,10
3,42367,8600000US42367,42367,2733647,1173,"POLYGON ((-87.17039 37.24358, -87.16652 37.243...",KY,42367,56,1,1
4,42345,8600000US42345,42345,395528652,4606061,"POLYGON ((-87.33103 37.20241, -87.33092 37.202...",KY,42345,54,1,1
5,42345,8600000US42345,42345,395528652,4606061,"POLYGON ((-87.33103 37.20241, -87.33092 37.202...",KY,42345,54,1,1
6,42718,8600000US42718,42718,653952872,14759412,"POLYGON ((-85.58518 37.52320, -85.58329 37.527...",KY,42718,677,1,1
7,42718,8600000US42718,42718,653952872,14759412,"POLYGON ((-85.58518 37.52320, -85.58329 37.527...",KY,42718,677,1,1
8,42718,8600000US42718,42718,653952872,14759412,"POLYGON ((-85.58518 37.52320, -85.58329 37.527...",KY,42718,677,1,1
9,42718,8600000US42718,42718,653952872,14759412,"POLYGON ((-85.58518 37.52320, -85.58329 37.527...",KY,42718,677,1,1


In [ ]:
# build final choropleth
mapbox = "INSERT MAPBOX STYLE LINK HERE"
mapbox_accesstoken="INSERT MAPBOX ACCESS TOKEN HERE"

# Create figure
fig = go.Figure()

# Add choropleth trace for zip code geometries
fig.add_trace(go.Choroplethmapbox(
    geojson=tn_map.__geo_interface__,
    locations=tn_map.index,
    z=tn_map['from_npi'],
    colorscale='Blues',
    marker_opacity=0.8,
    marker_line_width=0.5,
    customdata=pd.concat([tn_map['ZCTA5CE10'], tn_map['transaction_count']], axis=1),
    hovertemplate='Zip Code: %{customdata[0]}<br>' +
                  'Providers: %{z}<br>' +
                  'Total Transactions: %{customdata[1]}<extra></extra>',
    name='Zip Codes',
    colorbar=dict(
        title="Providers<br>By<br>TN<br>Zip<br>Code",
        thicknessmode="pixels", thickness=30,
        lenmode="pixels", len=500,
        yanchor="middle", y=0.5,
        tickfont=dict(size=13),
        ticks="outside", ticklen=5,
        tickcolor="#000",
        x=-0.12
    )
))

# Add choropleth trace for state geometries
fig.add_trace(go.Choroplethmapbox(
    geojson=state_map_tn.__geo_interface__,
    locations=state_map_tn.index,
    z=state_map_tn['referrals'],
    colorscale='Reds',
    marker_opacity=0.8,
    marker_line_width=0.5,
    customdata=pd.concat([state_map_tn['NAME'], state_map_tn['total_transactions']], axis=1),
    #text=state_map_tn['referrals'].astype(str),
    hovertemplate='<b>%{customdata[0]}</b><br>' +
                  'Providers: %{z}<br>' +
    'Total Transactions: %{customdata[1]}<extra></extra>',
    name='States',
    colorbar=dict(
        title="Providers<br>By<br>State",
        thicknessmode="pixels", thickness=30,
        lenmode="pixels", len=500,
        yanchor="middle", y=0.5,
        tickfont=dict(size=13),
        ticks="outside", ticklen=5,
        tickcolor="#000",
        x=1.01
    )
))

fig.add_trace(go.Choroplethmapbox(
    geojson=ky_map.__geo_interface__,
    locations=ky_map.index,
    z=ky_map['from_npi'],
    colorscale='purples',
    marker_opacity=0.8,
    marker_line_width=0.5,
    customdata=pd.concat([ky_map['ZCTA5CE10'], ky_map['transaction_count']], axis=1),
    hovertemplate='Zip Code: %{customdata[0]}<br>' +
                  'Providers: %{z}<br>' +
                  'Total Transactions: %{customdata[1]}<extra></extra>',
    name='Kentucky',
    colorbar=dict(
        title="Providers<br>By<br>KY Zip Code",
        thicknessmode="pixels", thickness=30,
        lenmode="pixels", len=500,
        xanchor="center", x=0.5, yanchor="bottom", y=0,
        tickfont=dict(size=13),
        ticks="outside", ticklen=5,
        tickcolor="#000",
        orientation="h"
    )
))

# Update layout
fig.update_layout(
    title='Out of State Nashville Providers',
    mapbox_style=mapbox,
    mapbox_zoom=2.9,
    mapbox_center={"lat": 39.8283, "lon": -98.5795},
    margin={"r":0,"t":0,"l":0,"b":0},
    #hovermode = False,
    #coloraxis_showscale=False,
    mapbox_accesstoken=mapbox_accesstoken,
    mapbox_layers=[{
        'below': 'traces',
        'sourcetype': 'geojson',
        'source': tn_map.__geo_interface__,
        'type': 'fill',
        'color': 'rgba(0,0,0,0)'
    }]
)

# Add text annotations for state geometries
for state, row in state_map_tn.iterrows():
    centroid = row.geometry.centroid
    fig.add_trace(
        go.Scattermapbox(
            lat=[centroid.y],
            lon=[centroid.x],
            text=str(row['referrals']),
            mode='text',
            textfont=dict(size=10, color='black'),
            showlegend=False,
            hoverinfo='none'
        )
    )
    
for zipcode, row in tn_map.iterrows():
    centroid = row.geometry.centroid
    fig.add_trace(
        go.Scattermapbox(
            lat=[centroid.y],
            lon=[centroid.x],
            text=str(row['from_npi']),
            mode='text',
            textfont=dict(size=10, color='black'),
            showlegend=False,
            hoverinfo='none'
        )
    )
    
for zipcode, row in ky_map.iterrows():
    centroid = row.geometry.centroid
    fig.add_trace(
        go.Scattermapbox(
            lat=[centroid.y],
            lon=[centroid.x],
            text=str(row['from_npi']),
            mode='text',
            textfont=dict(size=10, color='black'),
            showlegend=False,
            hoverinfo='none'
        )
    )

fig.show()
#fig.write_html('choropleth.html')